# This Notebook assesses model techniques of video classification analyzing different frames and algorithms.
This Notebook includes Explicit Imagery due to Pornography Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow import keras
#load weights
model = keras.models.load_model('/content/drive/MyDrive/PoleVsExplicit/saved models/pVe.h5')

In [ ]:
#Convert to frames
import cv2
import numpy as np
import os

def vidToFrames(path, Nthframe):
  vidcap = cv2.VideoCapture(path)

  frames = []

  #get nth frame
  n = Nthframe # this is nth
  it = 0 # this is for assessing if current frame = n

  # checks whether frames were extracted
  success = 1

  while success:
    # vidObj object calls read
    # function extract frames
    success, frame = vidcap.read()

    #if it isn't nth frame, continue
    if it != n:
      it += 1
      continue
    #else set 'iterator' to 0 and process frame
    else:
      it = 0
      try: #if image is BGR not RGB, then change
        imageRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      except:
        imageRGB = frame
      frames.append(imageRGB)

  vidcap.release()
  return frames

In [ ]:
def predictFrames(frames):
  predictions = []
  for frame in frames[0:-2]:
    img = cv2.resize(frame, (240, 240), interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    predictions.append(model.predict(img))
  #remove extra dimensions
  predictions = np.squeeze(predictions)
  return predictions

In [ ]:
from math import sqrt

#Favour explicit classifications at the beginning of a video
def begWeighted(predictions): 
  total = 0 #sum to be averaged
  for i in range (0,len(predictions)):
    #weighted prediction = prediction - (prediction * i/n predictions). Approaching the end, explicit prediction is penalized more.
    total += ((predictions[i] - (predictions[i] * i/len(predictions))))
  return total/len(predictions)

In [ ]:
#Favour explicit classifications at the end of a video
def endWeighted(predictions):
  total = 0 #sum to be averaged
  j = 0 #index increasing from 0

  for i in range (len(predictions)-1,-1, -1): #i = index going backwards
    #weighted prediction = prediction - prediction * i/n predictions. Approaching the beginning, explicit prediction is penalized more.
    total += ((predictions[i] - (predictions[i] * j/len(predictions))))
    j+=1
  return total/len(predictions) 

In [ ]:
#Favour explicit classifications at the centre of a video
def midWeighted(predictions):
  total = 0
  for i in range (0,len(predictions)):
    #weighted prediction = prediction - prediction * sqrt(((i+1)/n - 0.5)**2). Further from the center of predictions, explicit prediction is penalised more
    weight = sqrt((((i+1)/len(predictions)-0.5)**2))
    weight = predictions[i] - predictions[i] * weight
    total += weight
  return total/len(predictions)

In [ ]:
#analyse video
import matplotlib.pyplot as plt
def analyseVideo(path,NthFrame = 0):
  frames = vidToFrames(path,NthFrame)
  plt.imshow(frames[0])
  plt.show()
  print("Number of frames:", len(frames))

  predictions = predictFrames(frames)
  print("Mean classification:" ,np.average(predictions))
  print("Median classification:" ,np.median(predictions))
  begW = begWeighted(predictions)
  midW = midWeighted(predictions)
  endW = endWeighted(predictions)
  print("Beginning-Weighted Mean classification:" , begW)
  print("Mid-Weighted Mean classification:" , midW)
  print("End-Weighted Mean classification:" , endW)
  fullyWeighted = (begW + midW + endW)/3
  print("Fully-Weighted Mean classification:" , fullyWeighted)

In [ ]:
#Video 1 (Pole - Standard)
#Each video examined twice; every frame and at every 5th frame
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleStandardTest.mp4')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleStandardTest.mp4',5)

In [ ]:
#Video 2 (Pole- x2 Speed & Rotation)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/polex2Speed.mp4')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/polex2Speed.mp4',5)

In [ ]:
#Video 3 (Pole - Far Perspective)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleFarTest.mp4')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleFarTest.mp4',5)

In [ ]:
#Video 4 (Pole - Moving)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleMovingTest.mp4')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleMovingTest.mp4',5)

In [ ]:
#Video 5 (Pole - UV Lighting)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleUVTest.mp4')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/poleUVTest.mp4',5)

In [ ]:
#Video 6 & 7 (Porn- wsm)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/straight01.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/straight01.avi',5)

analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/straight02.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/straight02.avi',5)

In [ ]:
#Video 8 & 9 (Porn- msm)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/gay01.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/gay01.avi',5)

analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/gay02.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/gay02.avi',5)

In [ ]:
#Video 10 & 11 (Porn- wsw)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/lesbian01.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/lesbian01.avi',5)

analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/lesbian02.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/lesbian02.avi',5)

In [ ]:
#Video 12 & 13 (Porn- Cartoon wsm)
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/cartoon01.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/cartoon01.avi',5)

analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/cartoon02.avi')
analyseVideo('/content/drive/MyDrive/PoleVsExplicit/ClassifyVideos/cartoon02.avi',5)